In [52]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [53]:
DF_SMHM_data=pd.read_csv('Churn by Commitment Type_Full Data_data.csv')

In [40]:
DF_SMHM_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337506 entries, 0 to 2337505
Data columns (total 18 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   BACCT_GROUP_ID           int64  
 1   Bus Prod Instnc Id       int64  
 2   Snapshot Date            object 
 3   ADT_FLAG                 object 
 4   Bus Billing Account Num  int64  
 5   Business                 object 
 6   Commitment Type          object 
 7   Contract Expiring Soon   object 
 8   Contract Type            object 
 9   contract_type_group      object 
 10  DIY Activated            int64  
 11  FIFA Tag                 object 
 12  FIFA Commitment          int64  
 13  Acquisition Source       object 
 14  ACQUIRED_FROM            object 
 15  Aacquired_from_group     object 
 16  bus_inst_id_custom       float64
 17  province_state           object 
dtypes: float64(1), int64(5), object(12)
memory usage: 321.0+ MB


In [36]:
DF_SMHM_data.rename(columns={'bus_prod_instnc_id (Custom SQL Query)':'bus_inst_id_custom','PROVINCE_STATE_CD (Custom SQL Query)':'province_state','Contract Type (V4) (group)':'contract_type_group','Acquired_From (group)':'Aacquired_from_group'},inplace=True)

In [44]:
DF_SMHM_data.head()

,BACCT_GROUP_ID,Bus_Prod_Instnc_Id,Snapshot_Date,ADT_FLAG,Bus_Billing_Account_Num,Business,Commitment_Type,Contract_Expiring_Soon,Contract_Type,contract_type_group,DIY_Activated,FIFA_Tag,FIFA_Commitment,Acquisition_Source,ACQUIRED_FROM,Aacquired_from_group,bus_inst_id_custom,province_state
0,605737709,1136117829,11/30/2023 12:00:00 AM,N,605737709,Home,Contracted,Month-to-Month,Contracted,Off Contract,-1,Fifa,0,Acquired - ADT,NaN,Organic or TELUS,1.136118e+09,AB
1,38755328,1129887988,11/30/2023 12:00:00 AM,N,605361765,Home,MTM or Not-on-Contract,Not On Contract,Not On Contract,Off Contract,-1,Fifa,0,Organic,NaN,Organic or TELUS,1.129888e+09,QC
2,604703668,1115807709,1/31/2024 12:00:00 AM,N,604703668,Home,Contracted,Contracted,Contracted,Contracted,1,Fifa,0,Organic,NaN,Organic or TELUS,1.115808e+09,ON
3,235308095,1132620511,12/31/2023 12:00:00 AM,N,605505928,Home,Recontracted,Contracted,Recontracted,Contracted,-1,Fifa,0,Organic,NaN,Organic or TELUS,1.132621e+09,BC
4,600062383,1127635893,1/31/2024 12:00:00 AM,N,600062383,Home,Contracted,Contracted,Contracted,Contracted,-1,Fifa,0,TELUS,TELUS,Organic or TELUS,1.127636e+09,AB


In [43]:
DF_SMHM_data.columns = DF_SMHM_data.columns.str.replace(' ', '_')

In [46]:
DF_SMHM_data['Snapshot_Date'].value_counts()

Snapshot_Date
1/31/2024 12:00:00 AM     782603
12/31/2023 12:00:00 AM    780634
11/30/2023 12:00:00 AM    774269
Name: count, dtype: int64

In [37]:
# DF_SMHM_data.drop(['Contract Type (V4) (group)','Acquired_From (group)'],inplace=True,axis=1)

In [45]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'pras-pr-223186.pras_pr_dataset.SMHM_Base_Data'

bq_table_instance= bq_client.load_table_from_dataframe(DF_SMHM_data, Table_BQ,job_config=config)

In [15]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [16]:
# divg-team-v03-pr-de558a.shs_invol_churn.shs_invol_churn_base_data

In [47]:
Sales_order_Query='''


with original
as 
(SELECT 
min(dly_ord_itm_actvy_ts) as first_add,
bus_prod_instnc_id
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' and prod_typ_cd in ('SMHM') and ord_act_stat_cd = 'Processed' and bus_prod_instnc_id is not null
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01'

 group by bus_prod_instnc_id) /* Fetch original order date for the product instance */



,BAN_list as (

SELECT *  , 1 as Invol_churn
 FROM `pras-pr-223186.pras_pr_dataset.SHS_NEVERPAY_CHURN_TILLDOMO1_PRO26feb` 
 where PARSE_DATE("%d%b%Y", Churn_Date)>='2024-01-01' and PARSE_DATE("%d%b%Y", Churn_Date)<'2024-02-01' 

)

,BAN_Snapshot as (

select * from `pras-pr-223186.pras_pr_dataset.SMHM_Base_Data`
where Snapshot_Date='12/31/2023 12:00:00 AM'

)

,details as
(
SELECT 
dly_ord_itm_actvy_ts,
bus_billg_acct_num,
bus_prod_instnc_id,
prod_nm,
munic_nm,
prov_state_cd,
bi_chnl_tag_cd,
chnl_org_id,
prod_typ_cd,
src_sls_rep_cd,
src_typ_cd,
src_usr_chnl_txt,
txn_sub_typ_txt,
SLS_ACTVY_TXT
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' 
 --and prod_typ_cd in ('SMHM','HSIC','SING','TTV') 
 and prod_typ_cd in ('SMHM')
 and ord_act_stat_cd = 'Processed'
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01' 
 
 )
 
 
 
  /* Gather channel details */

,channel_refs as (


with
latest_update as (
SELECT
chnl_org_id,
max(chnl_org_key) as latest_key FROM `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim`
group by chnl_org_id

)

select distinct
t1.chnl_org_id,
chnl_org_txt from `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim` t1 inner join latest_update on latest_key = chnl_org_key

)

,Sales_Order_DF as (

select
details.bus_billg_acct_num as BAN_sales_order,
date(details.dly_ord_itm_actvy_ts) as Order_date,
details.bus_billg_acct_num as BAN,
cast (details.bus_prod_instnc_id as Integer) as Prod_Instnc_ID,
details.prod_nm as Prod_nm,
details.munic_nm,
details.prov_state_cd,
details.bi_chnl_tag_cd as Sales_Channel_tag,
details.chnl_org_id,
channel_refs.chnl_org_txt,
details.prod_typ_cd as Prod_Type,
details.src_sls_rep_cd as Sales_Agent_ID,
details.src_typ_cd,
details.src_usr_chnl_txt,
details.txn_sub_typ_txt,
details.SLS_ACTVY_TXT
from details inner join original on original.bus_prod_instnc_id = details.bus_prod_instnc_id and original.first_add = details.dly_ord_itm_actvy_ts  /* Inner join them to prevent duplicates */
 left join channel_refs on details.chnl_org_id = channel_refs.chnl_org_id /* left joining to get channel dealer names */
 where bi_chnl_tag_cd <> 'UNKNOWN' 
 order by details.dly_ord_itm_actvy_ts
)


select * 
from BAN_Snapshot a
left join BAN_list b
on a.Bus_Billing_Account_Num=b.BAN
left join Sales_Order_DF c
on a.Bus_Billing_Account_Num=c.BAN_sales_order and a.Bus_Prod_Instnc_Id=c.Prod_Instnc_ID






'''

In [48]:
DF_sales_Order=extract_bq_data(bq_client, sql=Sales_order_Query)

In [50]:
DF_sales_Order.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781290 entries, 0 to 781289
Data columns (total 105 columns):
 #    Column                   Dtype  
---   ------                   -----  
 0    BACCT_GROUP_ID           Int64  
 1    Bus_Prod_Instnc_Id       Int64  
 2    Snapshot_Date            object 
 3    ADT_FLAG                 object 
 4    Bus_Billing_Account_Num  Int64  
 5    Business                 object 
 6    Commitment_Type          object 
 7    Contract_Expiring_Soon   object 
 8    Contract_Type            object 
 9    contract_type_group      object 
 10   DIY_Activated            Int64  
 11   FIFA_Tag                 object 
 12   FIFA_Commitment          Int64  
 13   Acquisition_Source       object 
 14   ACQUIRED_FROM            object 
 15   Aacquired_from_group     object 
 16   bus_inst_id_custom       float64
 17   province_state           object 
 18   BAN                      Int64  
 19   CUSTOMER_ID              Int64  
 20   OBSERVATION_PROFILE_ID  

In [8]:
DF_sales_Order.head()

,BAN,CUSTOMER_ID,OBSERVATION_PROFILE_ID,BAN_STATUS,START_SERVICE_DT,TENURE_MTHS,TOTAL_BILLED_AMT,TREATABLE_AMT,NEVER_PAID_IND,BIRTH_DT,...,prov_state_cd,Sales_Channel_tag,chnl_org_id,chnl_org_txt,Prod_Type,Sales_Agent_ID,src_typ_cd,src_usr_chnl_txt,txn_sub_typ_txt,SLS_ACTVY_TXT
0,604371165,95975532,109113479,O,27Aug2020,39,63.00,0.00,0,09Apr1991,...,AB,CARE_SMHM_FO,-1000000000,UNKNOWN,SMHM,t873927,FIFA,Retention Pilot Agents,New Contract,N/AVAIL
1,604865874,103300066,125734272,O,23Sep2021,27,722.41,628.02,0,23Mar1965,...,ON,CSS,-1000000000,UNKNOWN,SMHM,x233166,FIFA,CSS,New Contract,CSS
2,604865874,103300066,125734272,O,23Sep2021,27,722.41,628.02,0,23Mar1965,...,UKN,CARE-WLN,-1000000000,UNKNOWN,SMHM,t873927,FIFA,CLS,New Contract,N/AVAIL
3,605143666,104058629,133670256,O,12Apr2022,20,366.36,260.00,0,31Dec1974,...,BC,CARE-WLN,-1000000000,UNKNOWN,SMHM,t928775,FIFA,Frontline,New Contract,N/AVAIL
4,605341350,104568969,139378025,O,10Aug2022,16,236.38,174.64,0,17Jun1960,...,UKN,D2C,12073883,Go Sales,SMHM,UKN,FIFA,"ICI171 - ON (21 KENVIEW BLVD BRAMPTON, ON L6T5...",New Contract,N/AVAIL


In [51]:
DF_sales_Order.to_csv('SMHM_Jan2024_Base_data.csv',index=False)